In [1]:
import import_ipynb
import pickle
import numpy as np
import torch
import torch.nn as nn
from model import GTN
from utils import f1_score

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from utils.ipynb


In [2]:
epochs = 40
node_dim = 64
num_channels = 2
lr = 0.005
weight_decay = 0.001
num_layers = 2
norm = True
adaptive_lr = True

In [3]:
with open('data/ACM/node_features.pkl', 'rb') as f:
    node_features = pickle.load(f)

In [4]:
node_features

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [5]:
node_features.shape

(8994, 1902)

In [6]:
with open('data/ACM/edges.pkl', 'rb') as f:
    edges = pickle.load(f)

In [7]:
edges

[<8994x8994 sparse matrix of type '<class 'numpy.int32'>'
 	with 9936 stored elements in Compressed Sparse Row format>,
 <8994x8994 sparse matrix of type '<class 'numpy.int32'>'
 	with 9936 stored elements in Compressed Sparse Column format>,
 <8994x8994 sparse matrix of type '<class 'numpy.int32'>'
 	with 3025 stored elements in Compressed Sparse Row format>,
 <8994x8994 sparse matrix of type '<class 'numpy.int32'>'
 	with 3025 stored elements in Compressed Sparse Column format>]

In [8]:
with open('data/ACM/labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [9]:
labels

[array([[ 826,    0],
        [1823,    0],
        [1382,    0],
        ...,
        [2480,    2],
        [2596,    2],
        [2958,    2]], dtype=int64),
 array([[1770,    0],
        [ 888,    0],
        [1737,    0],
        [1793,    0],
        [2389,    0],
        [1454,    0],
        [1600,    0],
        [2374,    0],
        [1067,    0],
        [1125,    0],
        [ 963,    0],
        [1551,    0],
        [ 991,    0],
        [1358,    0],
        [1101,    0],
        [1913,    0],
        [1677,    0],
        [1540,    0],
        [1909,    0],
        [1038,    0],
        [1042,    0],
        [1452,    0],
        [1560,    0],
        [ 979,    0],
        [2361,    0],
        [1837,    0],
        [1568,    0],
        [1941,    0],
        [1815,    0],
        [1258,    0],
        [1751,    0],
        [1655,    0],
        [1859,    0],
        [1569,    0],
        [1756,    0],
        [2352,    0],
        [1583,    0],
        [1260,    0],
    

In [10]:
num_nodes = edges[0].shape[0]

In [11]:
for i, edge in enumerate(edges):
    if i == 0:
        A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
    else:
        A = torch.cat([A, torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
A = torch.cat([A, torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)

In [12]:
A

tensor([[[0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         

In [13]:
A.shape

torch.Size([8994, 8994, 5])

In [14]:
node_features = torch.from_numpy(node_features).type(torch.FloatTensor)
train_node = torch.from_numpy(np.array(labels[0])[:, 0]).type(torch.LongTensor)
train_target = torch.from_numpy(np.array(labels[0])[:, 1]).type(torch.LongTensor)
valid_node = torch.from_numpy(np.array(labels[1])[:, 0]).type(torch.LongTensor)
valid_target = torch.from_numpy(np.array(labels[1])[:, 1]).type(torch.LongTensor)
test_node = torch.from_numpy(np.array(labels[2])[:, 0]).type(torch.LongTensor)
test_target = torch.from_numpy(np.array(labels[2])[:, 1]).type(torch.LongTensor)

In [15]:
num_classes = torch.max(train_target).item() + 1
final_f1 = 0

In [16]:
model = GTN(num_edge=A.shape[-1],
            num_channels=num_channels,
            w_in=node_features.shape[1],
            w_out=node_dim,
            num_class=num_classes,
            num_layers=num_layers,
            norm=norm)
if adaptive_lr == 'false':
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
else:
    optimizer = torch.optim.Adam([{'params': model.weight},
                                  {'params': model.linear1.parameters()},
                                  {'params': model.linear2.parameters()},
                                  {"params": model.layers.parameters(), "lr": 0.5}
                                  ], lr=0.005, weight_decay=0.001)

In [17]:
model

GTN(
  (layers): ModuleList(
    (0): GTLayer(
      (conv1): GTConv()
      (conv2): GTConv()
    )
    (1): GTLayer(
      (conv1): GTConv()
    )
  )
  (loss): CrossEntropyLoss()
  (linear1): Linear(in_features=128, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=3, bias=True)
)

In [18]:
loss = nn.CrossEntropyLoss()
# Train & Valid
best_train_loss = 10000
best_val_loss = 10000
best_train_f1 = 0
best_val_f1 = 0
best_epoch = 0

In [19]:
for i in range(epochs):
    for param_group in optimizer.param_groups:
        if param_group['lr'] > 0.005:
            param_group['lr'] = param_group['lr'] * 0.9
    print('Epoch: ', i + 1)
    model.zero_grad()
    model.train()
    loss, y_train, Ws = model(A, node_features, train_node, train_target)
    train_f1 = torch.mean(
        f1_score(torch.argmax(y_train.detach(), dim=1), train_target, num_classes=num_classes)).cpu().numpy()
    print('Train_Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))
    loss.backward()
    optimizer.step()
    # Valid
    model.eval()
    with torch.no_grad():
        val_loss, y_valid, _ = model.forward(A, node_features, valid_node, valid_target)
        val_f1 = torch.mean(
            f1_score(torch.argmax(y_valid, dim=1), valid_target, num_classes=num_classes)).cpu().numpy()
        print('Valid_Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
    if val_f1 > best_val_f1:
        best_train_loss = loss.detach().cpu().numpy()
        best_val_loss = val_loss.detach().cpu().numpy()
        best_train_f1 = train_f1
        best_val_f1 = val_f1
        best_epoch = i + 1
        torch.save(model.state_dict(), "model_pth/best.pth")

Epoch:  1
Train_Loss: 1.1074516773223877, Macro_F1: 0.18466687202453613
Valid_Loss: 1.0783792734146118, Macro_F1: 0.4778386652469635
Epoch:  2
Train_Loss: 1.076841950416565, Macro_F1: 0.521491289138794
Valid_Loss: 1.010503888130188, Macro_F1: 0.710550844669342
Epoch:  3
Train_Loss: 1.0047333240509033, Macro_F1: 0.7918422818183899
Valid_Loss: 0.8797913789749146, Macro_F1: 0.8900139927864075
Epoch:  4
Train_Loss: 0.8665217161178589, Macro_F1: 0.9247635006904602
Valid_Loss: 0.7209909558296204, Macro_F1: 0.8014833331108093
Epoch:  5
Train_Loss: 0.7036378979682922, Macro_F1: 0.86018967628479
Valid_Loss: 0.6231492161750793, Macro_F1: 0.8641390800476074
Epoch:  6
Train_Loss: 0.5907576084136963, Macro_F1: 0.9215969443321228
Valid_Loss: 0.532245934009552, Macro_F1: 0.8286805152893066
Epoch:  7
Train_Loss: 0.5095663666725159, Macro_F1: 0.8583758473396301
Valid_Loss: 0.427819162607193, Macro_F1: 0.8594194054603577
Epoch:  8
Train_Loss: 0.3975590467453003, Macro_F1: 0.9062349200248718
Valid_Loss: 

In [22]:
#Test
model.load_state_dict(torch.load("model_pth/best.pth"))
model.eval()
print('----------------Test----------------')
with torch.no_grad():
    best_test_loss, y_test, W = model.forward(A, node_features, test_node, test_target)
    best_test_f1 = torch.mean(f1_score(torch.argmax(y_test, dim=1), test_target, num_classes=num_classes)).cpu().numpy()
print('Best Epoch: ', best_epoch)
print('Train_Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
print('Valid_Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
print('Test_Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))

----------------Test----------------
Best Epoch:  18
Train_Loss: 0.0670320987701416, Macro_F1: 0.9732419848442078
Valid_Loss: 0.20265190303325653, Macro_F1: 0.9401242733001709
Test_Loss: 0.23855070769786835, Macro_F1: 0.9202559590339661
